In [1]:
import asyncio
import aiokafka # requires kafka version >= 9 
from kafka import KafkaProducer, KafkaConsumer

topic='test_topic_1'

In [2]:
# Try just reading one record from Kafka
consumer = KafkaConsumer(topic,auto_offset_reset='earliest', enable_auto_commit=False,consumer_timeout_ms=100, value_deserializer=lambda x: x.decode('utf-8'))
next(consumer).value

'Hello, World'

In [3]:
# examples from https://github.com/aio-libs/aiokafka

from kafka.common import KafkaError
from aiokafka import AIOKafkaConsumer

@asyncio.coroutine
def consume_task(consumer):
    while True:
        try:
            msg = yield from consumer.getone()
            print("consumed: ", msg.topic, msg.partition, msg.offset,
                  msg.key, msg.value, msg.timestamp)
        except KafkaError as err:
            print("error while consuming message: ", err)

loop = asyncio.get_event_loop()
consumer = AIOKafkaConsumer(
    'test_topic_1', loop=loop)
#    'topic1', 'topic2', loop=loop, bootstrap_servers='localhost:1234')
# Bootstrap client, will get initial cluster metadata
loop.run_until_complete(consumer.start())
c_task = loop.create_task(consume_task(consumer))
try:
    loop.run_forever()
finally:
    # Will gracefully leave consumer group; perform autocommit if enabled
    loop.run_until_complete(consumer.stop())
    c_task.cancel()
    #loop.close()


Task exception was never retrieved
future: <Task finished coro=<wait_for() done, defined at /home/egor/anaconda3/lib/python3.5/asyncio/tasks.py:355> exception=CorrelationIdError('Correlation ids do not match: sent 1, recv 2',)>
Traceback (most recent call last):
  File "/home/egor/anaconda3/lib/python3.5/asyncio/tasks.py", line 239, in _step
    result = coro.send(None)
  File "/home/egor/anaconda3/lib/python3.5/asyncio/tasks.py", line 392, in wait_for
    return fut.result()
  File "/home/egor/anaconda3/lib/python3.5/asyncio/futures.py", line 274, in result
    raise self._exception
kafka.errors.CorrelationIdError: CorrelationIdError: Correlation ids do not match: sent 1, recv 2


KeyboardInterrupt: 

In [4]:
import asyncio
from aiokafka import AIOKafkaProducer

@asyncio.coroutine
def produce(loop):
    # Just adds message to sending queue
    future = yield from producer.send('foobar', b'some_message_bytes')
    # waiting for message to be delivered
    resp = yield from future
    print("Message produced: partition {}; offset {}".format(
          resp.partition, resp.offset))
    # Also can use a helper to send and wait in 1 call
    resp = yield from producer.send_and_wait(
        'foobar', key=b'foo', value=b'bar')
    #resp = yield from producer.send_and_wait(
    #   'foobar', b'message for partition 1', partition=1)

loop = asyncio.get_event_loop()
producer = AIOKafkaProducer(loop=loop, bootstrap_servers='localhost:9092')
# Bootstrap client, will get initial cluster metadata
loop.run_until_complete(producer.start())
loop.run_until_complete(produce(loop))
# Wait for all pending messages to be delivered or expire
loop.run_until_complete(producer.stop())
#loop.close()

Task exception was never retrieved
future: <Task finished coro=<wait_for() done, defined at /home/egor/anaconda3/lib/python3.5/asyncio/tasks.py:355> exception=CorrelationIdError('Correlation ids do not match: sent 1, recv 2',)>
Traceback (most recent call last):
  File "/home/egor/anaconda3/lib/python3.5/asyncio/tasks.py", line 239, in _step
    result = coro.send(None)
  File "/home/egor/anaconda3/lib/python3.5/asyncio/tasks.py", line 392, in wait_for
    return fut.result()
  File "/home/egor/anaconda3/lib/python3.5/asyncio/futures.py", line 274, in result
    raise self._exception
kafka.errors.CorrelationIdError: CorrelationIdError: Correlation ids do not match: sent 1, recv 2


Message produced: partition 0; offset 8
